In [36]:
from ObservationObject.Observation import Observation
import airmass_ct as air
import astrometry_ct as asm
import constants_ct as cst
import database_ct as db
import errors_ct as ers
import fits_utils_ct as fut
import pending_ct as pd
import utils_ct as ut

# DEBUG-ONLY
from importlib import reload
mods = [asm, cst, db, ers, fut, ut, pd]
for mod in mods:
    reload(mod)
    
#---------------------------------------------------#

import argparse
from datetime import datetime
import os
import sys
import time
import warnings

def load_args():
    """ Function that specifies the expected arguments
        given on the command line by the user.
    """
    
    # Sets the description that will be shown with --help
    parser = argparse.ArgumentParser(description="Program that updates the Blaauw Observatory \
                                                  database by automatically reducing the fits files")
    
    # Add required argument for path to observation folder
    target = parser.add_mutually_exclusive_group()
    target.add_argument("-f", "--folder", type=str, help="The absolute path to the observation directory")
    target.add_argument("-d", "--date", type=str, help="The observation date in yy-mm-dd")
    target.add_argument("-u", "--update", action='store_const', const=datetime.today().strftime("%y-%m-%d"),
                        dest="date", help="Select todays directory if it exists")
    
    # Add arguments for the intended action
    parser.add_argument("-b", "--backup", action="store_true", help="Move a copy of the raw data to local folder")
    parser.add_argument("-s", "--saveraw", action="store_true", help="Save raw correction frames locally")
    parser.add_argument("-r", "--reduce", action="store_true", help="Reduce all the present light frames")
    parser.add_argument("-a", "--astrometry", action="store_true", help="Run all light frames through Astrometry")
    #parser.add_argument("-p", "--preview", action="store_true", help="Create a .png preview per light file")
    parser.add_argument("-p", "--pending", action="store_true", help="Rerun pending reductions")
    parser.add_argument("-m", "--metadata", action="store_true", help="Add extra header information (airmass etc.)")
    
    # Add argument for verbosity
    parser.add_argument("-v", "--verbose", action="store_true", help="Increase the amount of output")
    
    # Retrieve the passed arguments and check their validity
    args = parser.parse_args()
    targets = check_args(parser, args)
    
    return args, targets

def check_args(parser, args):
    """ Function that checks all the given arguments for their validity. If one
        of the required conditions is not met, an error is thrown.
    """
    # Check that a target was specified
    if not (args.folder or args.date):
        parser.error(f"Target was not specified: pass a directory path or a date")
    
    # If the target is a directory, get the datasets it contains
    if args.folder:
        if not os.path.isdir(args.folder):
            parser.error(f"{args.folder} is not an existing directory")
        targets = ut.get_targets_from_folder(args.folder)
    
    # If the target is a date, get the datasets this date contains
    elif args.date:
        if not ut.is_valid_date(args.date, "%y-%m-%d"):
            parser.error(f"{args.date} is not a valid date. Use the following format: yy-mm-dd")
        targets = ut.get_targets_from_date(args.date)
    
    # The action was not specified
    if not (args.backup or args.saveraw or args.reduce or args.pending or args.astrometry):
        parser.error("No action specified. Possible actions include: --backup, --reduce, --pending and --astrometry")
        
    # Update the user on the found contents
    if len(targets) > 0:
        ut.Print(f"Found data in {len(targets)} subdir(s)", args, True)
    else:
        ut.Print("No data found for this target", args, True)
        # sys.exit() met script
        
    return targets
                
def check_obs(obs, args):
    """ Function that checks some properties of the generated 
        Observation object. Nothing fancy, just some extra checks 
        to prevent unforeseen circumstances...
    """
    # Update the user on the found content
    if args.verbose:
        ut.Print(f"Found {len(obs.files)} fits files", args)
        ut.Print(f"Found {len(obs.biasFiles)} bias frames", args)
        ut.Print(f"Found {len(obs.darkFiles)} dark frames", args)
        ut.Print(f"Found {len(obs.flatFiles)} flat fields", args)
        ut.Print(f"Found {len(obs.lightFiles)} light frames", args)
    
    # For reducing, we also need the raw correction frames
#     if args.reduce:
#         args.saveraw = True
     
    # Now, check what correction frame types we have
    frame_check = 3
    
    # Check for the presence of bias frames
    bias_found = len(obs.biasFiles) > 0
    if not bias_found:
        warnings.warn("No bias files were found")
        frame_check -= 1
        
    # Check for dark presence of frames    
    dark_found = len(obs.darkFiles) > 0
    if not dark_found:
        warnings.warn("No dark files were found")
        frame_check -= 1
    
    # Check for flat presence of fields
    flat_found = len(obs.flatFiles) > 0
    if not flat_found:
        warnings.warn("No flat fields were found")
        frame_check -= 1
        
    # Not likely to happen, but raise an error if
    # literally no correction frames were found
    if frame_check == 0:
        raise ers.MissingFramesError("No suitable correction frames found")
    
    return

def get_observation(target, args):
    """ Function that creates the observation object for the 
        passed target. Also performs some small checkups to 
        be sure that we have some proper data.
    """
    ut.Print("Creating Observation object...", args)
    obs = Observation(target)  
    check_obs(obs, args)
    ut.Print("Observation object initialized", args, True)
    return obs

def run_backup(obs, working_dir, args):
    ut.Print("Creating backup...", args, True)
    db.create_backup(obs, working_dir, args)
    ut.Print("Backup created!", args, True)
    
def run_saveraw(obs, working_dir, args):
    ut.Print("Saving correction frames...", args, True)
    fut.save_correction(obs, working_dir, args)
    ut.Print("Correction frames saved!", args, True)

def run_reduce(obs, working_dir, args):
    ut.Print("Reducing images...", args, True)
    fut.reduce_imgs(obs, working_dir, args)
    ut.Print("Images reduced!", args, True)
    
def run_pending(obs, working_dir, args):
    ut.Print("Rerunning pending reductions...", args, True)
    pd.rerun_pending(obs, working_dir, args)
    ut.Print("Pending list updated!", args, True)

def run_metadata(obs, working_dir, args):
    ut.Print("Adding header metadata...", args, True)
#     air.add_airmass(obs.lightFiles, working_dir, args)
    # Add other header keywords...
    ut.Print("Added metadata", args, True)
    
def run_astrometry(obs, working_dir, args):
    ut.Print("Running astrometry client...", args, True)
    asm.start_astrometry(obs, working_dir, args)
    ut.Print("Astrometry run complete!", args, True)
            
def main():
    # Retrieve the arguments, and the specified targets
    args, targets = load_args()
    
    # Loop over every target that was found and perform the actions 
    # that were specified in the command line arguments
    for target in targets:
        ut.Print(f"Currently handling: {target}", args, True)
        
        # Construct a working dir where all *new/altered* data goes
        working_dir_name = os.path.relpath(target, cst.tele_path)
        working_dir = os.path.join(cst.base_path, working_dir_name)
        
        # Extra check if the working dir already exists, else create it
        if os.path.isdir(working_dir):
            # TODO: this means this data probably has been handled already...
            # BUT: not sure what exact operations have been carried out
            # Maybe we can put some sort of log in each handled dir, containing
            # a list of all actions that were already performed? Seems better 
            # than just 'checking' manually what has or hasn't been done yet.
            #print("Folder existed")
            pass
        else:
            os.makedirs(working_dir)
        
        # Create observation object for this target
        obs = get_observation(target, args)
    
        # Perform actions specified by user
        if args.backup: run_backup(obs, working_dir, args)
        if args.saveraw: run_saveraw(obs, working_dir, args)
        if args.reduce: run_reduce(obs, working_dir, args)
        if args.pending: run_pending(obs, working_dir, args)
        if args.metadata: run_metadata(obs, working_dir, args)
        if args.astrometry: run_astrometry(obs, working_dir, args)
    
if __name__ == "__main__":
    # placeholder
    # data_dir = "/net/vega/data/users/observatory/images/200417/STL-6303E/i/"
    date = "20-04-20"
    #date = "20-05-05"
    #22 april
    
    sys.argv = ["wrapper.py", "-d", date, "-pv"]
#    sys.argv = ["wrapper.py", "-d", date, "-bv"]
#     sys.argv = ["wrapper.py", "-u", "-rv"]
    main()

15:31:47.480780 Found data in 1 subdir(s)
15:31:47.480882 Currently handling: /net/vega/data/users/observatory/images/200420/STL-6303E/i
15:31:47.482495 Creating Observation object...
15:31:47.913375 Found 123 fits files
15:31:47.913528 Found 27 bias frames
15:31:47.913589 Found 4 dark frames
15:31:47.913640 Found 15 flat fields
15:31:47.913690 Found 77 light frames
15:31:47.913741 Observation object initialized
15:31:47.913793 Rerunning pending reductions...
Current line: ['2020-04-22' 'Dark file' '1x1' 'R' '-4' '-' '-' '2020-04-26'
 '/net/dataserver3/data/users/sterrenwacht/obslog/200422/STL-6303E/i/Correction/master_dark1x1C1.fits']
15:31:49.941129 /net/dataserver3/data/users/sterrenwacht/obslog/200422/STL-6303E/i/Correction/master_dark1x1C1.fits created
Current line: ['2020-04-22' 'Light file' '3x3' 'O[III]' '?' '?' '?' '-'
 '/net/vega/data/users/observatory/images/200422/STL-6303E/i/200422_Li_.00000116.NGC_6543.FIT']
Current line: ['2020-04-22' 'Light file' '3x3' 'S[II]' '?' '?' '